In [1]:
# installing required libraries
import selenium 
from selenium import webdriver
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from itertools import repeat
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common import exceptions 
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
#loading the driver
driver=webdriver.Chrome('chromedriver.exe')

In [3]:
#defining attach function and empty lists that are required
car_name_list=[]
sub_model_list=[]
km_driven_list=[]
fuel_type_list=[]
gear_type_list=[]
amount_list=[]
car_type_list=[]
location_list=[]
def attach(x,y):
    for i in x:
        y.append(i.text)

In [4]:
#type of the data that need to be collected
city_locations=['ahmedabad','bangalore','chennai','hyderabad','jaipur','kolkata','mumbai','new-delhi']
cars_types=['suv','sedan','hatchback','muv','minivan','luxury']

In [5]:
#extracting the data from the website
for i in city_locations:
    for j in cars_types:
        url='https://www.cardekho.com/used-'+j+'+cars+in+'+i
        driver.get(url)
        
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-2000);")

            # Wait to load page
            time.sleep(4)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(4)
        car_name=driver.find_elements_by_xpath('//div[@class="gsc_col-xs-7  carsName"]/a')
        sub_model=driver.find_elements_by_xpath('//div[@class="gsc_col-xs-7  carsName"]/div[1]')
        km_driven=driver.find_elements_by_xpath('//div[@class="gsc_col-xs-7  carsName"]/div[2]/span[1]')
        fuel_type=driver.find_elements_by_xpath('//div[@class="gsc_col-xs-7  carsName"]/div[2]/span[2]')
        gear_type=driver.find_elements_by_xpath('//div[@class="truncate dotlist"]/span[3]')
        amount=driver.find_elements_by_xpath('//span[@class="amnt "]')
        attach(car_name,car_name_list)
        attach(sub_model,sub_model_list)
        attach(km_driven,km_driven_list)
        attach(fuel_type,fuel_type_list)
        attach(gear_type,gear_type_list)
        attach(amount,amount_list)
        car_type_list.extend(repeat(j,len(car_name)))
        location_list.extend(repeat(i,len(car_name)))

In [6]:
#framing the data
Car_dataset=pd.DataFrame(list(zip(car_name_list,sub_model_list,km_driven_list,fuel_type_list,gear_type_list,amount_list,car_type_list,location_list)))
Car_dataset.columns=['Car Name','Model','Km Driven','Fuel type','Gear Type','Price','Car Type','Location']
Car_dataset.index+=1
Car_dataset

,Car Name,Model,Km Driven,Fuel type,Gear Type,Price,Car Type,Location
1,2015 Hyundai Creta,1.6 CRDi SX Option,"75,428 kms",Diesel,Manual,"8,10,000",suv,ahmedabad
2,2020 Mahindra XUV500,W7,"8,898 kms",Diesel,Manual,"14,40,000",suv,ahmedabad
3,2020 MG Hector,Sharp MT,"28,835 kms",Petrol,Manual,"17,40,000",suv,ahmedabad
4,2019 Hyundai Creta,1.6 CRDi SX,"55,144 kms",Diesel,Manual,"13,10,000",suv,ahmedabad
5,2018 Renault Duster,Petrol RXS CVT,"32,937 kms",Petrol,Automatic,"7,20,000",suv,ahmedabad
...,...,...,...,...,...,...,...,...
10043,2013 Maruti Eeco,Flexi Green,"2,00,000 kms",CNG,Manual,"2,00,000",minivan,new-delhi
10044,2018 Volvo XC90,D5 Inscription BSIV,"39,500 kms",Diesel,Automatic,"59,50,000",luxury,new-delhi
10045,2020 BMW X6,Xdrive40i M Sport,"4,000 kms",Petrol,Automatic,"1,05,00,000",luxury,new-delhi
10046,2018 Volvo XC90,D5 Inscription BSIV,"57,000 kms",Diesel,Automatic,"57,50,000",luxury,new-delhi


In [7]:
#saving the collected data from the website in excel format
file_name = 'Car_Pricing_Data.xlsx'
Car_dataset.to_excel(file_name)